In [ ]:
!pip install SimpleITK

     |████████████████████████████████| 48.4 MB 2.2 MB/s 


In [ ]:
import os
from os import path
import nibabel as nib
from nibabel.testing import data_path
import matplotlib.pyplot as plt
import cv2
from google.colab.patches import cv2_imshow
import numpy as np
import sys
from glob import glob
import pandas as pd
import tensorflow as tf
import copy
from tqdm import tqdm
import SimpleITK as sitk
from random import random

In [ ]:
!pip install nipype

In [ ]:
!pip install nilearn

     |████████████████████████████████| 10.1 MB 6.7 MB/s 


In [ ]:
from nipype.interfaces.ants import N4BiasFieldCorrection

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
img=nib.load('/content/drive/MyDrive/NFBS_Dataset/A00028185/sub-A00028185_ses-NFB3_T1w.nii.gz')
print('Shape of image=',img.shape)

Shape of image= (256, 256, 192)


In [ ]:
#storing the address of 3 types of files
brain_mask=[]
brain=[]
raw=[]
for subdir, dirs, files in os.walk('/content/drive/MyDrive/NFBS_Dataset'):
    for file in files:
        filepath = subdir + os.sep + file
        if filepath.endswith(".gz"):
          if '_brainmask.' in filepath:
            brain_mask.append(filepath)
          elif '_brain.' in filepath:
            brain.append(filepath)
          else:
            raw.append(filepath)

In [ ]:
df = pd.DataFrame()
df['raw'] = raw
df['brain'] = brain
df['brain_mask'] = brain_mask
df.head()

,raw,brain,brain_mask
0,/content/drive/MyDrive/NFBS_Dataset/A00063326/...,/content/drive/MyDrive/NFBS_Dataset/A00063326/...,/content/drive/MyDrive/NFBS_Dataset/A00063326/...
1,/content/drive/MyDrive/NFBS_Dataset/A00062917/...,/content/drive/MyDrive/NFBS_Dataset/A00062917/...,/content/drive/MyDrive/NFBS_Dataset/A00062917/...
2,/content/drive/MyDrive/NFBS_Dataset/A00064081/...,/content/drive/MyDrive/NFBS_Dataset/A00064081/...,/content/drive/MyDrive/NFBS_Dataset/A00064081/...
3,/content/drive/MyDrive/NFBS_Dataset/A00062942/...,/content/drive/MyDrive/NFBS_Dataset/A00062942/...,/content/drive/MyDrive/NFBS_Dataset/A00062942/...
4,/content/drive/MyDrive/NFBS_Dataset/A00062351/...,/content/drive/MyDrive/NFBS_Dataset/A00062351/...,/content/drive/MyDrive/NFBS_Dataset/A00062351/...


In [ ]:
len(df)

125

In [ ]:
from nilearn.image import resample_img

In [ ]:
df1 = pd.DataFrame()

class preprocessing():
  def __init__(self,df):
    self.data=df
    self.raw_index=[]
    self.mask_index=[]
    
    
  def n4biasfield_correction(self):
    path = '/content/drive/MyDrive/NFBS augmented data'
    if(os.path.exists(path)== False):
      os.mkdir('/content/drive/MyDrive/NFBS augmented data')
      os.mkdir('/content/drive/MyDrive/NFBS augmented data/processed')
    path = '/content/drive/MyDrive/NFBS augmented data/processed' 

    raw1 = []
    for i in range(len(self.data)):
      img_path = self.data.raw.iloc[i]
      mask_path = self.data.brain_mask.iloc[i]
      
      image = sitk.ReadImage(img_path, sitk.sitkFloat32)
      maskImage = sitk.ReadImage(mask_path, sitk.sitkUInt8)
    
      corrector = sitk.N4BiasFieldCorrectionImageFilter()
      numberFittingLevels = 4
      maxIter = 10
      corrector.SetMaximumNumberOfIterations([maxIter] * numberFittingLevels)

      corrected_image = corrector.Execute(image, maskImage)

      new_path = os.path.join(path,'rawbias'+str(i)+'.nii.gz')
      raw1.append(new_path)

      sitk.WriteImage(corrected_image, new_path)
    
    print("done appending")
    df1['raw'] = raw1
    df1['brain'] = df['brain']
    df1['brain_mask'] = df['brain_mask']

    self.data = df1
    self.raw_index = df1['raw']
    print('Bias correction done')
    return


  '''def resize_crop(self):
    #Reducing the size of image due to memory constraints
    path = '/content/drive/MyDrive/NFBS augmented data/processed/'
    target_shape = np.array((96,128,160))                   #reducing size of image from 256*256*192 to 96*128*160
    new_resolution = [2,]*3
    new_affine = np.zeros((4,4))
    new_affine[:3,:3] = np.diag(new_resolution)

    # putting point 0,0,0 in the middle of the new volume - this could be refined in the future

    new_affine[:3,3] = target_shape*new_resolution/2.*-1
    new_affine[3,3] = 1.
    #print(new_affine)

    #resizing both image and mask and storing in folder

    for i in range(len(self.data)):
        downsampled_and_cropped_nii = resample_img(self.data.raw.iloc[i], target_affine=new_affine, target_shape=target_shape, interpolation='nearest')
        filename = os.path.join(path+'rawbias'+str(i)+'.nii.gz')
        downsampled_and_cropped_nii.to_filename(filename)
        filename1 = filename.replace('bias','crop')
        os.rename(filename,filename1)
        self.raw_index.append(path+'rawcrop'+str(i)+'.nii.gz')
        
        downsampled_and_cropped_nii = resample_img(self.data.brain_mask.iloc[i], target_affine=new_affine, target_shape=target_shape, interpolation='nearest')
        filename = os.path.join(path+'maskcrop'+str(i)+'.nii.gz')
        downsampled_and_cropped_nii.to_filename(filename)
        filename1 = filename.replace('bias','crop')
        os.rename(filename,filename1)
        self.mask_index.append(path+'maskcrop'+str(i)+'.nii.gz')

    print('Cropping done')
    return   '''


  def intensity_normalization(self):
    path = '/content/drive/MyDrive/NFBS augmented data/processed'
    for i in self.raw_index:
      #print(i)
      image = sitk.ReadImage(i)
      resacleFilter = sitk.RescaleIntensityImageFilter()
      resacleFilter.SetOutputMaximum(255)
      resacleFilter.SetOutputMinimum(0)
      image = resacleFilter.Execute(image)
      sitk.WriteImage(image,i)
      j= i.replace('bias','norm')
      os.rename(i,j)
    print('Normalization done')
    return

In [ ]:
obj = preprocessing(df)
#obj.n4biasfield_correction()
obj.resize_crop()
obj.intensity_normalization()

Cropping done
/content/drive/MyDrive/NFBS augmented data/processed/rawcrop0.nii.gz
/content/drive/MyDrive/NFBS augmented data/processed/rawcrop1.nii.gz
/content/drive/MyDrive/NFBS augmented data/processed/rawcrop2.nii.gz
/content/drive/MyDrive/NFBS augmented data/processed/rawcrop3.nii.gz
/content/drive/MyDrive/NFBS augmented data/processed/rawcrop4.nii.gz
/content/drive/MyDrive/NFBS augmented data/processed/rawcrop5.nii.gz
/content/drive/MyDrive/NFBS augmented data/processed/rawcrop6.nii.gz
/content/drive/MyDrive/NFBS augmented data/processed/rawcrop7.nii.gz
/content/drive/MyDrive/NFBS augmented data/processed/rawcrop8.nii.gz
/content/drive/MyDrive/NFBS augmented data/processed/rawcrop9.nii.gz
/content/drive/MyDrive/NFBS augmented data/processed/rawcrop10.nii.gz
/content/drive/MyDrive/NFBS augmented data/processed/rawcrop11.nii.gz
/content/drive/MyDrive/NFBS augmented data/processed/rawcrop12.nii.gz
/content/drive/MyDrive/NFBS augmented data/processed/rawcrop13.nii.gz
/content/drive/M

In [ ]:
img=nib.load('/content/drive/MyDrive/NFBS augmented data/processed/raw0.nii.gz')
n1_header = img.header
print(n1_header)

<class 'nibabel.nifti1.Nifti1Header'> object, endian='<'
sizeof_hdr      : 348
data_type       : b''
db_name         : b''
extents         : 0
session_error   : 0
regular         : b''
dim_info        : 0
dim             : [  3 256 256 192   1   1   1   1]
intent_p1       : 0.0
intent_p2       : 0.0
intent_p3       : 0.0
intent_code     : none
datatype        : float32
bitpix          : 32
slice_start     : 0
pixdim          : [1. 2. 2. 2. 1. 1. 1. 1.]
vox_offset      : 0.0
scl_slope       : nan
scl_inter       : nan
slice_end       : 0
slice_code      : unknown
xyzt_units      : 0
cal_max         : 0.0
cal_min         : 0.0
slice_duration  : 0.0
toffset         : 0.0
glmax           : 0
glmin           : 0
descrip         : b''
aux_file        : b''
qform_code      : unknown
sform_code      : aligned
quatern_b       : 0.0
quatern_c       : 0.0
quatern_d       : 0.0
qoffset_x       : -256.0
qoffset_y       : -256.0
qoffset_z       : -192.0
srow_x          : [   2.    0.    0. -256.]
sr

In [ ]:
import shutil
directory = '/content/drive/MyDrive/NFBS augmented data/processed'

raw_path =  '/content/drive/MyDrive/NFBS augmented data/processed/raw'
if(os.path.exists(raw_path)== False):
  os.mkdir('/content/drive/MyDrive/NFBS augmented data/processed/raw')

mask_path = '/content/drive/MyDrive/NFBS augmented data/processed/mask'
if(os.path.exists(mask_path)== False):
  os.mkdir('/content/drive/MyDrive/NFBS augmented data/processed/mask')

for filename in os.listdir(directory):
  f = os.path.join(directory, filename)
  # checking if it is a file
  if os.path.isfile(f):
      if('raw' in f):
        shutil.move(f, raw_path)
      elif('mask' in f):
        shutil.move(f, mask_path) 

In [ ]:
import albumentations as A
import tensorflow as tf
import tensorflow.keras.backend as K
from tensorflow.keras.losses import binary_crossentropy

from tensorflow.keras.models import *
from tensorflow.keras.layers import *
from tensorflow.keras.optimizers import *
from tensorflow.keras.callbacks import ModelCheckpoint, ReduceLROnPlateau, CSVLogger

import math

In [ ]:
import random
def data_gen(img_list, mask_list, batch_size):
    '''Custom data generator to feed image to model'''
    c = 0
    n = [i for i in range(len(img_list))]  #List of training images
    random.shuffle(n)
    
    while (True):
      img = np.zeros((batch_size, 96, 128, 160,1)).astype('float')   #adding extra dimensions as conv3d takes file of size 5
      mask = np.zeros((batch_size, 96, 128, 160,1)).astype('float')

      for i in range(c, c+batch_size): 
        train_img = nib.load(img_list[n[i]]).get_data()
        train_img=np.expand_dims(train_img,-1)

        train_mask = nib.load(mask_list[n[i]]).get_data()
        train_mask=np.expand_dims(train_mask,-1)

        '''augmented = train_transform(image= train_img, mask= train_mask)
        train_img = augmented ['image']
        train_mask = augmented ['mask']
        train_img = image.astype(np.float32)
        train_mask = np.repeat(image[..., np.newaxis], 3, -1)  '''
        train_img = train_img.astype(np.float32)
        train_mask = train_mask.astype(np.float32)

        #print(train_img, " ", train_mask)

        img[i-c]=train_img
        mask[i-c] = train_mask
      c+=batch_size
      if(c+batch_size>=len(img_list)):
        c=0
        random.shuffle(n)

    yield img,mask

In [ ]:
data_gen(train_x,train_y,5)

<generator object data_gen at 0x7fd3d4226f50>

In [ ]:
train_gen = data_gen(train_x,train_y, batch_size = 5)

In [ ]:
# Keras implementation of the paper:
# 3D MRI Brain Tumor Segmentation Using Autoencoder Regularization
# by Myronenko A. (https://arxiv.org/pdf/1810.11654.pdf)
# Author of this code: Suyog Jadhav (https://github.com/IAmSUyogJadhav)

import keras.backend as K
from keras.losses import mse
from keras.layers import Conv3D, Activation, Add, UpSampling3D, Lambda, Dense
from keras.layers import Input, Reshape, Flatten, Dropout, SpatialDropout3D
from tensorflow.keras.optimizers import Adam
from keras.models import Model
try:
    from group_norm import GroupNormalization
except ImportError:
    import urllib.request
    print('Downloading group_norm.py in the current directory...')
    url = 'https://raw.githubusercontent.com/titu1994/Keras-Group-Normalization/master/group_norm.py'
    urllib.request.urlretrieve(url, "group_norm.py")
    from group_norm import GroupNormalization


def green_block(inp, filters, data_format='channels_first', name=None):
    """
    green_block(inp, filters, name=None)
    ------------------------------------
    Implementation of the special residual block used in the paper. The block
    consists of two (GroupNorm --> ReLu --> 3x3x3 non-strided Convolution)
    units, with a residual connection from the input `inp` to the output. Used
    internally in the model. Can be used independently as well.
    Parameters
    ----------
    `inp`: An keras.layers.layer instance, required
        The keras layer just preceding the green block.
    `filters`: integer, required
        No. of filters to use in the 3D convolutional block. The output
        layer of this green block will have this many no. of channels.
    `data_format`: string, optional
        The format of the input data. Must be either 'chanels_first' or
        'channels_last'. Defaults to `channels_first`, as used in the paper.
    `name`: string, optional
        The name to be given to this green block. Defaults to None, in which
        case, keras uses generated names for the involved layers. If a string
        is provided, the names of individual layers are generated by attaching
        a relevant prefix from [GroupNorm_, Res_, Conv3D_, Relu_, ], followed
        by _1 or _2.
    Returns
    -------
    `out`: A keras.layers.Layer instance
        The output of the green block. Has no. of channels equal to `filters`.
        The size of the rest of the dimensions remains same as in `inp`.
    """
    inp_res = Conv3D(
        filters=filters,
        kernel_size=(1, 1, 1),
        strides=1,
        data_format=data_format,
        name=f'Res_{name}' if name else None)(inp)

    # axis=1 for channels_first data format
    # No. of groups = 8, as given in the paper
    x = GroupNormalization(
        groups=8,
        axis=1 if data_format == 'channels_first' else 0,
        name=f'GroupNorm_1_{name}' if name else None)(inp)
    x = Activation('relu', name=f'Relu_1_{name}' if name else None)(x)
    x = Conv3D(
        filters=filters,
        kernel_size=(3, 3, 3),
        strides=1,
        padding='same',
        data_format=data_format,
        name=f'Conv3D_1_{name}' if name else None)(x)

    x = GroupNormalization(
        groups=8,
        axis=1 if data_format == 'channels_first' else 0,
        name=f'GroupNorm_2_{name}' if name else None)(x)
    x = Activation('relu', name=f'Relu_2_{name}' if name else None)(x)
    x = Conv3D(
        filters=filters,
        kernel_size=(3, 3, 3),
        strides=1,
        padding='same',
        data_format=data_format,
        name=f'Conv3D_2_{name}' if name else None)(x)

    out = Add(name=f'Out_{name}' if name else None)([x, inp_res])
    return out


# From keras-team/keras/blob/master/examples/variational_autoencoder.py
def sampling(args):
    """Reparameterization trick by sampling from an isotropic unit Gaussian.
    # Arguments
        args (tensor): mean and log of variance of Q(z|X)
    # Returns
        z (tensor): sampled latent vector
    """
    z_mean, z_var = args
    batch = K.shape(z_mean)[0]
    dim = K.int_shape(z_mean)[1]
    # by default, random_normal has mean = 0 and std = 1.0
    epsilon = K.random_normal(shape=(batch, dim))
    return z_mean + K.exp(0.5 * z_var) * epsilon


def dice_coefficient(y_true, y_pred):
    intersection = K.sum(K.abs(y_true * y_pred), axis=[-3,-2,-1])
    dn = K.sum(K.square(y_true) + K.square(y_pred), axis=[-3,-2,-1]) + 1e-8
    return K.mean(2 * intersection / dn, axis=[0,1])


def loss_gt(e=1e-8):
    """
    loss_gt(e=1e-8)
    ------------------------------------------------------
    Since keras does not allow custom loss functions to have arguments
    other than the true and predicted labels, this function acts as a wrapper
    that allows us to implement the custom loss used in the paper. This function
    only calculates - L<dice> term of the following equation. (i.e. GT Decoder part loss)
    
    L = - L<dice> + weight_L2 ∗ L<L2> + weight_KL ∗ L<KL>
    
    Parameters
    ----------
    `e`: Float, optional
        A small epsilon term to add in the denominator to avoid dividing by
        zero and possible gradient explosion.
        
    Returns
    -------
    loss_gt_(y_true, y_pred): A custom keras loss function
        This function takes as input the predicted and ground labels, uses them
        to calculate the dice loss.
        
    """
    def loss_gt_(y_true, y_pred):
        intersection = K.sum(K.abs(y_true * y_pred), axis=[-3,-2,-1])
        dn = K.sum(K.square(y_true) + K.square(y_pred), axis=[-3,-2,-1]) + e
        
        return - K.mean(2 * intersection / dn, axis=[0,1])
    
    return loss_gt_

def loss_VAE(input_shape, z_mean, z_var, weight_L2=0.1, weight_KL=0.1):
    """
    loss_VAE(input_shape, z_mean, z_var, weight_L2=0.1, weight_KL=0.1)
    ------------------------------------------------------
    Since keras does not allow custom loss functions to have arguments
    other than the true and predicted labels, this function acts as a wrapper
    that allows us to implement the custom loss used in the paper. This function
    calculates the following equation, except for -L<dice> term. (i.e. VAE decoder part loss)
    
    L = - L<dice> + weight_L2 ∗ L<L2> + weight_KL ∗ L<KL>
    
    Parameters
    ----------
     `input_shape`: A 4-tuple, required
        The shape of an image as the tuple (c, H, W, D), where c is
        the no. of channels; H, W and D is the height, width and depth of the
        input image, respectively.
    `z_mean`: An keras.layers.Layer instance, required
        The vector representing values of mean for the learned distribution
        in the VAE part. Used internally.
    `z_var`: An keras.layers.Layer instance, required
        The vector representing values of variance for the learned distribution
        in the VAE part. Used internally.
    `weight_L2`: A real number, optional
        The weight to be given to the L2 loss term in the loss function. Adjust to get best
        results for your task. Defaults to 0.1.
    `weight_KL`: A real number, optional
        The weight to be given to the KL loss term in the loss function. Adjust to get best
        results for your task. Defaults to 0.1.
        
    Returns
    -------
    loss_VAE_(y_true, y_pred): A custom keras loss function
        This function takes as input the predicted and ground labels, uses them
        to calculate the L2 and KL loss.
        
    """
    def loss_VAE_(y_true, y_pred):
        c, H, W, D = input_shape
        n = c * H * W * D
        
        loss_L2 = K.mean(K.square(y_true - y_pred), axis=(1, 2, 3, 4)) # original axis value is (1,2,3,4).

        loss_KL = (1 / n) * K.sum(
            K.exp(z_var) + K.square(z_mean) - 1. - z_var,
            axis=-1
        )

        return weight_L2 * loss_L2 + weight_KL * loss_KL

    return loss_VAE_

def build_model(input_shape=(4, 160, 192, 128), output_channels=3, weight_L2=0.1, weight_KL=0.1, dice_e=1e-8):
    """
    build_model(input_shape=(4, 160, 192, 128), output_channels=3, weight_L2=0.1, weight_KL=0.1)
    -------------------------------------------
    Creates the model used in the BRATS2018 winning solution
    by Myronenko A. (https://arxiv.org/pdf/1810.11654.pdf)
    Parameters
    ----------
    `input_shape`: A 4-tuple, optional.
        Shape of the input image. Must be a 4D image of shape (c, H, W, D),
        where, each of H, W and D are divisible by 2^4, and c is divisible by 4.
        Defaults to the crop size used in the paper, i.e., (4, 160, 192, 128).
    `output_channels`: An integer, optional.
        The no. of channels in the output. Defaults to 3 (BraTS 2018 format).
    `weight_L2`: A real number, optional
        The weight to be given to the L2 loss term in the loss function. Adjust to get best
        results for your task. Defaults to 0.1.
    `weight_KL`: A real number, optional
        The weight to be given to the KL loss term in the loss function. Adjust to get best
        results for your task. Defaults to 0.1.
    `dice_e`: Float, optional
        A small epsilon term to add in the denominator of dice loss to avoid dividing by
        zero and possible gradient explosion. This argument will be passed to loss_gt function.
    Returns
    -------
    `model`: A keras.models.Model instance
        The created model.
    """
    c, H, W, D = input_shape
    assert len(input_shape) == 4, "Input shape must be a 4-tuple"
    assert (c % 4) == 0, "The no. of channels must be divisible by 4"
    assert (H % 16) == 0 and (W % 16) == 0 and (D % 16) == 0, \
        "All the input dimensions must be divisible by 16"


    # -------------------------------------------------------------------------
    # Encoder
    # -------------------------------------------------------------------------

    ## Input Layer
    inp = Input(input_shape)

    ## The Initial Block
    x = Conv3D(
        filters=32,
        kernel_size=(3, 3, 3),
        strides=1,
        padding='same',
        data_format='channels_first',
        name='Input_x1')(inp)

    ## Dropout (0.2)
    x = SpatialDropout3D(0.2, data_format='channels_first')(x)

    ## Green Block x1 (output filters = 32)
    x1 = green_block(x, 32, name='x1')
    x = Conv3D(
        filters=32,
        kernel_size=(3, 3, 3),
        strides=2,
        padding='same',
        data_format='channels_first',
        name='Enc_DownSample_32')(x1)

    ## Green Block x2 (output filters = 64)
    x = green_block(x, 64, name='Enc_64_1')
    x2 = green_block(x, 64, name='x2')
    x = Conv3D(
        filters=64,
        kernel_size=(3, 3, 3),
        strides=2,
        padding='same',
        data_format='channels_first',
        name='Enc_DownSample_64')(x2)

    ## Green Blocks x2 (output filters = 128)
    x = green_block(x, 128, name='Enc_128_1')
    x3 = green_block(x, 128, name='x3')
    x = Conv3D(
        filters=128,
        kernel_size=(3, 3, 3),
        strides=2,
        padding='same',
        data_format='channels_first',
        name='Enc_DownSample_128')(x3)

    ## Green Blocks x4 (output filters = 256)
    x = green_block(x, 256, name='Enc_256_1')
    x = green_block(x, 256, name='Enc_256_2')
    x = green_block(x, 256, name='Enc_256_3')
    x4 = green_block(x, 256, name='x4')

    # -------------------------------------------------------------------------
    # Decoder
    # -------------------------------------------------------------------------

    ## GT (Groud Truth) Part
    # -------------------------------------------------------------------------

    ### Green Block x1 (output filters=128)
    x = Conv3D(
        filters=128,
        kernel_size=(1, 1, 1),
        strides=1,
        data_format='channels_first',
        name='Dec_GT_ReduceDepth_128')(x4)
    x = UpSampling3D(
        size=2,
        data_format='channels_first',
        name='Dec_GT_UpSample_128')(x)
    x = Add(name='Input_Dec_GT_128')([x, x3])
    x = green_block(x, 128, name='Dec_GT_128')

    ### Green Block x1 (output filters=64)
    x = Conv3D(
        filters=64,
        kernel_size=(1, 1, 1),
        strides=1,
        data_format='channels_first',
        name='Dec_GT_ReduceDepth_64')(x)
    x = UpSampling3D(
        size=2,
        data_format='channels_first',
        name='Dec_GT_UpSample_64')(x)
    x = Add(name='Input_Dec_GT_64')([x, x2])
    x = green_block(x, 64, name='Dec_GT_64')

    ### Green Block x1 (output filters=32)
    x = Conv3D(
        filters=32,
        kernel_size=(1, 1, 1),
        strides=1,
        data_format='channels_first',
        name='Dec_GT_ReduceDepth_32')(x)
    x = UpSampling3D(
        size=2,
        data_format='channels_first',
        name='Dec_GT_UpSample_32')(x)
    x = Add(name='Input_Dec_GT_32')([x, x1])
    x = green_block(x, 32, name='Dec_GT_32')

    ### Blue Block x1 (output filters=32)
    x = Conv3D(
        filters=32,
        kernel_size=(3, 3, 3),
        strides=1,
        padding='same',
        data_format='channels_first',
        name='Input_Dec_GT_Output')(x)

    ### Output Block
    out_GT = Conv3D(
        filters=output_channels,  # No. of tumor classes is 3
        kernel_size=(1, 1, 1),
        strides=1,
        data_format='channels_first',
        activation='sigmoid',
        name='Dec_GT_Output')(x)

    ## VAE (Variational Auto Encoder) Part
    # -------------------------------------------------------------------------

    ### VD Block (Reducing dimensionality of the data)
    x = GroupNormalization(groups=8, axis=1, name='Dec_VAE_VD_GN')(x4)
    x = Activation('relu', name='Dec_VAE_VD_relu')(x)
    x = Conv3D(
        filters=16,
        kernel_size=(3, 3, 3),
        strides=2,
        padding='same',
        data_format='channels_first',
        name='Dec_VAE_VD_Conv3D')(x)

    # Not mentioned in the paper, but the author used a Flattening layer here.
    x = Flatten(name='Dec_VAE_VD_Flatten')(x)
    x = Dense(256, name='Dec_VAE_VD_Dense')(x)

    ### VDraw Block (Sampling)
    z_mean = Dense(128, name='Dec_VAE_VDraw_Mean')(x)
    z_var = Dense(128, name='Dec_VAE_VDraw_Var')(x)
    x = Lambda(sampling, name='Dec_VAE_VDraw_Sampling')([z_mean, z_var])

    ### VU Block (Upsizing back to a depth of 256)
    x = Dense((c//4) * (H//16) * (W//16) * (D//16))(x)
    x = Activation('relu')(x)
    x = Reshape(((c//4), (H//16), (W//16), (D//16)))(x)
    x = Conv3D(
        filters=256,
        kernel_size=(1, 1, 1),
        strides=1,
        data_format='channels_first',
        name='Dec_VAE_ReduceDepth_256')(x)
    x = UpSampling3D(
        size=2,
        data_format='channels_first',
        name='Dec_VAE_UpSample_256')(x)

    ### Green Block x1 (output filters=128)
    x = Conv3D(
        filters=128,
        kernel_size=(1, 1, 1),
        strides=1,
        data_format='channels_first',
        name='Dec_VAE_ReduceDepth_128')(x)
    x = UpSampling3D(
        size=2,
        data_format='channels_first',
        name='Dec_VAE_UpSample_128')(x)
    x = green_block(x, 128, name='Dec_VAE_128')

    ### Green Block x1 (output filters=64)
    x = Conv3D(
        filters=64,
        kernel_size=(1, 1, 1),
        strides=1,
        data_format='channels_first',
        name='Dec_VAE_ReduceDepth_64')(x)
    x = UpSampling3D(
        size=2,
        data_format='channels_first',
        name='Dec_VAE_UpSample_64')(x)
    x = green_block(x, 64, name='Dec_VAE_64')

    ### Green Block x1 (output filters=32)
    x = Conv3D(
        filters=32,
        kernel_size=(1, 1, 1),
        strides=1,
        data_format='channels_first',
        name='Dec_VAE_ReduceDepth_32')(x)
    x = UpSampling3D(
        size=2,
        data_format='channels_first',
        name='Dec_VAE_UpSample_32')(x)
    x = green_block(x, 32, name='Dec_VAE_32')

    ### Blue Block x1 (output filters=32)
    x = Conv3D(
        filters=32,
        kernel_size=(3, 3, 3),
        strides=1,
        padding='same',
        data_format='channels_first',
        name='Input_Dec_VAE_Output')(x)

    ### Output Block
    out_VAE = Conv3D(
        filters=4,
        kernel_size=(1, 1, 1),
        strides=1,
        data_format='channels_first',
        name='Dec_VAE_Output')(x) 

    # Build and Compile the model
    out = out_GT
    model = Model(inp, outputs=[out, out_VAE])  # Create the model
    model.compile(
        adam(lr=1e-4),
        [loss_gt(dice_e), loss_VAE(input_shape, z_mean, z_var, weight_L2=weight_L2, weight_KL=weight_KL)],
        metrics=[dice_coefficient]
    )

    return model